In [30]:
import sys
sys.path.append('../')

In [31]:
from utils.utils import Utils
from utils.preprocess import Preprocess
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
from textblob import TextBlob

import numpy as np
import pandas as pd

In [32]:
%%time
utils = Utils()
pre = Preprocess(mode="clean")

CPU times: user 2.9 s, sys: 80 ms, total: 2.98 s
Wall time: 2.99 s


In [42]:
columns = ['message_id', 'response_id', 'article_id', 'gold_empathy', 'gold_empathy_bin', 'gold_distress', 'gold_distress_bin', 'gold_emotion', 'essay', 'gender', 'education', 'race', 'age', 'income', 'personality_conscientiousness', 'personality_openess', 'personality_extraversion', 'personality_agreeableness', 'personality_stability', 'iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern']
len(columns)

23

In [43]:
%%time
train_data = pd.read_csv("../dataset/train-final.csv", usecols=columns)
print(train_data.shape)
dev_data = pd.read_csv("../dataset/dev-final.csv", usecols=columns)
print(dev_data.shape)

(1860, 23)
(270, 23)
CPU times: user 118 ms, sys: 8.12 ms, total: 126 ms
Wall time: 124 ms


In [37]:
train_gold = pd.read_csv("")

28

In [ ]:
train_data = train_data[]

In [6]:
def categorize_income(income):
    if income >0 and income <=25000:
        return 0
    elif income >25000 and income <=75000:
        return 1
    elif income >75000:
        return 2

In [7]:
train_data['age_bin'] = train_data.age.apply(lambda x: utils.categorize_age(x))
dev_data['age_bin'] = dev_data.age.apply(lambda x: utils.categorize_age(x))

In [8]:
train_data['income_bin'] = train_data.income.apply(lambda x: categorize_income(x))
dev_data['income_bin'] = dev_data.income.apply(lambda x: categorize_income(x))

In [9]:
train_data[train_data.gold_distress_bin == 1].gold_emotion.value_counts()

sadness     353
anger       161
neutral     123
fear         95
disgust      84
surprise     63
joy          26
Name: gold_emotion, dtype: int64

In [10]:
train_empathy = train_data.gold_empathy.values.tolist()
dev_empathy = dev_data.gold_empathy.values.tolist()

In [11]:
train_distress = train_data.gold_distress.values.tolist()
dev_distress = dev_data.gold_distress.values.tolist()

In [12]:
# ----------------------------------- Function to get word emotion and vad scores -----------------------------------
            
def get_word_scores(sentiment='anger'):
    path = "../resources/NRC-resources/"+sentiment+"-scores.txt"
    word_scores = {}
    with open(path) as f:
        data = f.readlines()
        for row in data:
            row = row.split("\t")
            row[1] = row[1].split("\n")
            word_scores[row[0]] = row[1][0]
    f.close()
    return word_scores

In [13]:
# -------------------------------------------- Function to get essay emotion and vad scores --------------------------------------------
    
def get_essay_nrc_scores(essay, nrc_features, normalize=False):
    word_scores_list = []
    for element in nrc_features:
        word_scores = get_word_scores(element)
        word_scores_list.append(word_scores)
    essay_scores = np.zeros((len(essay), len(word_scores_list)))

    for i in range(len(essay)):        
        for j in range(len(word_scores_list)):
            score = 0
            cnt = 0
            for word in essay[i].split():
                if word in word_scores_list[j].keys():
                    cnt = cnt = 1
                    score = score + float(word_scores_list[j].get(word))
            essay_scores[i][j] = score

    return essay_scores

In [15]:
def column(matrix, i):
    return [row[i] for row in matrix]

# Polarity and Subjectivity Correlation

In [16]:
train_pol_sub_score = [TextBlob(text).sentiment[0:] for text in train_data.essay.values.tolist()]
train_pol_sub_score = np.reshape(train_pol_sub_score, (len(train_pol_sub_score), len(train_pol_sub_score[0])))
print(train_pol_sub_score.shape)

dev_pol_sub_score = [TextBlob(text).sentiment[0:] for text in dev_data.essay.values.tolist()]
dev_pol_sub_score = np.reshape(dev_pol_sub_score, (len(dev_pol_sub_score), len(dev_pol_sub_score[0])))
print(dev_pol_sub_score.shape)

(1860, 2)
(270, 2)


In [17]:
print("\nEmpathy Correlation with Training Data: \n")
for i in range(train_pol_sub_score.shape[1]):
    print("correlation: {}".format(pearsonr(column(train_pol_sub_score, i), train_empathy)))

print("\nEmpathy Correlation with Dev Data: \n")
for i in range(dev_pol_sub_score.shape[1]):
    print("correlation: {}".format(pearsonr(column(dev_pol_sub_score, i), dev_empathy)))


Empathy Correlation with Training Data: 

correlation: (0.00389210599831934, 0.866783546646021)
correlation: (-0.004917572529348033, 0.8321519825511748)

Empathy Correlation with Dev Data: 

correlation: (-0.08488273878797196, 0.1642842771437893)
correlation: (0.08407187709229258, 0.16836621527448983)


In [18]:
print("\nDistress Correlation with Training Data: \n")
for i in range(train_pol_sub_score.shape[1]):
    print("correlation: {}".format(pearsonr(column(train_pol_sub_score, i), train_distress)))

print("\nDistress Correlation with Dev Data: \n")
for i in range(dev_pol_sub_score.shape[1]):
    print("correlation: {}".format(pearsonr(column(dev_pol_sub_score, i), dev_distress)))


Distress Correlation with Training Data: 

correlation: (-0.12335056260856325, 9.468298582176119e-08)
correlation: (0.018079898334861736, 0.43581228877306216)

Distress Correlation with Dev Data: 

correlation: (-0.1926336062066933, 0.0014704774194548625)
correlation: (0.13647598422513407, 0.024920801710967763)


# NRC features

In [19]:
def get_feature_correlation(feature_list, train_empathy, dev_empathy, train_df, dev_df, remove_stopwords=False, lemmatize=False):
    scaler = StandardScaler()
    
    train_essay = [pre.clean_text(text, remove_stopwords=remove_stopwords, lemmatize=lemmatize) for text in train_df.essay.values.tolist()]
    train_emotion_scores = get_essay_nrc_scores(train_essay,
                                                feature_list, 
                                                normalize=False)
    train_emotion_scores = scaler.fit_transform(train_emotion_scores)
    
    print("\nResult on Training Data: ")
    for i in range(train_emotion_scores.shape[1]):
        print("NRC feature {} correlation: {}".format(feature_list[i], 
                                                      pearsonr(column(train_emotion_scores, i), train_empathy)))
    
    dev_essay = [pre.clean_text(text, remove_stopwords=remove_stopwords, lemmatize=lemmatize) for text in dev_df.essay.values.tolist()]
    dev_emotion_scores = get_essay_nrc_scores(dev_essay,
                                              feature_list, 
                                              normalize=False)
    dev_emotion_scores = scaler.transform(dev_emotion_scores)
    print("\nResult on Dev Data: ")
    for i in range(dev_emotion_scores.shape[1]):
        print("NRC feature {} correlation: {}".format(feature_list[i],
                                                      pearsonr(column(dev_emotion_scores, i), dev_empathy)))

In [20]:
nrc_features = ['anger', 
                'anticipation',
                'disgust',
                'fear',
                'joy',
                'sadness',
                'surprise',
                'trust',
                'valence',
                'arousal',
                'dominance']

# Empathy Correlation with NRC features

In [21]:
get_feature_correlation(nrc_features,
                        train_empathy,
                        dev_empathy,
                        train_data,
                        dev_data,
                        remove_stopwords=False,
                        lemmatize=False)


Result on Training Data: 
NRC feature anger correlation: (0.03519546491704102, 0.12917840725953858)
NRC feature anticipation correlation: (0.04855615947291328, 0.03626499794891488)
NRC feature disgust correlation: (0.03374312719421774, 0.14575233699284404)
NRC feature fear correlation: (0.11529751923632318, 6.17023657049862e-07)
NRC feature joy correlation: (0.12350147220368538, 9.130842900773421e-08)
NRC feature sadness correlation: (0.18750409649239852, 3.5323211326586747e-16)
NRC feature surprise correlation: (-0.023379658509559482, 0.31356431158103415)
NRC feature trust correlation: (0.03246616708188888, 0.1616273382459173)
NRC feature valence correlation: (0.1132201738225091, 9.810939223932556e-07)
NRC feature arousal correlation: (0.10828796224764228, 2.8569748302895547e-06)
NRC feature dominance correlation: (0.10395695970475859, 7.0362391753387335e-06)

Result on Dev Data: 
NRC feature anger correlation: (0.024186977842458837, 0.6923657590680352)
NRC feature anticipation corre

# Distress Correlation with NRC features

In [22]:
get_feature_correlation(nrc_features,
                        train_distress,
                        dev_distress,
                        train_data,
                        dev_data,
                        remove_stopwords=False,
                        lemmatize=False)


Result on Training Data: 
NRC feature anger correlation: (0.15769918055900028, 7.948622968698792e-12)
NRC feature anticipation correlation: (0.048901341896994126, 0.034957777249386694)
NRC feature disgust correlation: (0.14563771199062264, 2.781850737936384e-10)
NRC feature fear correlation: (0.2152478815447958, 6.164474907658634e-21)
NRC feature joy correlation: (0.018120403529394882, 0.43478461272636293)
NRC feature sadness correlation: (0.20203771193252357, 1.3871719196226824e-18)
NRC feature surprise correlation: (0.002190605777983197, 0.9247815433649075)
NRC feature trust correlation: (0.044911928324245405, 0.052791317893546526)
NRC feature valence correlation: (0.0889515526390839, 0.0001223769943598398)
NRC feature arousal correlation: (0.15344894073077817, 2.873909888717811e-11)
NRC feature dominance correlation: (0.12388119389289999, 8.332394455680527e-08)

Result on Dev Data: 
NRC feature anger correlation: (0.24162105209629256, 6.0356117685540976e-05)
NRC feature anticipatio

# IRI and Personality Features

In [23]:
iri_features = ['iri_perspective_taking', 'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern']
personality_features = ['personality_conscientiousness', 'personality_openess', 'personality_extraversion', 'personality_agreeableness', 'personality_stability']

In [24]:
train_iri = train_data[iri_features].values
print(train_iri.shape)

dev_iri = dev_data[iri_features].values
print(dev_iri.shape)

(1860, 4)
(270, 4)


In [25]:
train_personality = train_data[personality_features].values
print(train_personality.shape)

dev_personality = dev_data[personality_features].values
print(dev_personality.shape)

(1860, 5)
(270, 5)


# Empathy Correlation with IRI Features

In [26]:
print("\nEmpathy correlation on Training Data: \n")
for i in range(train_iri.shape[1]):
    print("NRC feature {} correlation: {}".format(iri_features[i], 
                                                  pearsonr(column(train_iri, i), train_empathy)))
    
print("\nEmpathy correlation on Dev Data: \n")
for i in range(dev_iri.shape[1]):
    print("NRC feature {} correlation: {}".format(iri_features[i], 
                                                  pearsonr(column(dev_iri, i), dev_empathy)))


Empathy correlation on Training Data: 

NRC feature iri_perspective_taking correlation: (0.22856663040279532, 1.8023870796887119e-23)
NRC feature iri_personal_distress correlation: (0.005806115549177553, 0.8024030801218004)
NRC feature iri_fantasy correlation: (0.16171192259347203, 2.2863423583074164e-12)
NRC feature iri_empathatic_concern correlation: (0.2993633343983349, 8.080200148943306e-40)

Empathy correlation on Dev Data: 

NRC feature iri_perspective_taking correlation: (0.202989418359576, 0.0007937146200273498)
NRC feature iri_personal_distress correlation: (0.04738842311180669, 0.43804758577495945)
NRC feature iri_fantasy correlation: (0.15504622031303647, 0.010732314263238226)
NRC feature iri_empathatic_concern correlation: (0.24955210765314614, 3.363615805989282e-05)


# Empathy Correlation with Big5 Features

In [27]:
print("\nEmpathy correlation on Training Data: \n")
for i in range(train_personality.shape[1]):
    print("NRC feature {} correlation: {}".format(personality_features[i], 
                                                  pearsonr(column(train_personality, i), train_empathy)))
    
print("\nEmpathy correlation on Dev Data: \n")
for i in range(dev_personality.shape[1]):
    print("NRC feature {} correlation: {}".format(personality_features[i], 
                                                  pearsonr(column(dev_personality, i), dev_empathy)))


Empathy correlation on Training Data: 

NRC feature personality_conscientiousness correlation: (0.09263106209527383, 6.31060186467026e-05)
NRC feature personality_openess correlation: (0.11377269271127469, 8.679270957892684e-07)
NRC feature personality_extraversion correlation: (0.20902561681934412, 8.272935808223537e-20)
NRC feature personality_agreeableness correlation: (0.24325722930340982, 1.8525600705176216e-26)
NRC feature personality_stability correlation: (0.1037886896228663, 7.281898804278378e-06)

Empathy correlation on Dev Data: 

NRC feature personality_conscientiousness correlation: (0.05585836172304273, 0.36055805782871764)
NRC feature personality_openess correlation: (0.20788348445686353, 0.0005867699857419374)
NRC feature personality_extraversion correlation: (-0.07142000579522055, 0.24216201875575047)
NRC feature personality_agreeableness correlation: (0.2388727885280508, 7.357726509917349e-05)
NRC feature personality_stability correlation: (-0.07024350772260109, 0.25

# Distress Correlation with IRI Features

In [28]:
print("\nDistress correlation on Training Data: \n")
for i in range(train_iri.shape[1]):
    print("NRC feature {} correlation: {}".format(iri_features[i], 
                                                  pearsonr(column(train_iri, i), train_distress)))
    
print("\nDistress correlation on Dev Data: \n")
for i in range(dev_iri.shape[1]):
    print("NRC feature {} correlation: {}".format(iri_features[i], 
                                                  pearsonr(column(dev_iri, i), dev_distress)))


Distress correlation on Training Data: 

NRC feature iri_perspective_taking correlation: (0.12652424670336937, 4.373892590075604e-08)
NRC feature iri_personal_distress correlation: (0.18952486622501458, 1.678101271410058e-16)
NRC feature iri_fantasy correlation: (0.1205412837507924, 1.84614008033545e-07)
NRC feature iri_empathatic_concern correlation: (0.19864478258616805, 5.256970035825554e-18)

Distress correlation on Dev Data: 

NRC feature iri_perspective_taking correlation: (0.17102091893068003, 0.004833700017363015)
NRC feature iri_personal_distress correlation: (0.3755251101638157, 1.8084889333538824e-10)
NRC feature iri_fantasy correlation: (0.33916964288043544, 1.0801312513148551e-08)
NRC feature iri_empathatic_concern correlation: (0.23107911176029347, 0.00012742451421095185)


# Distress Correlation with Big5 Features

In [29]:
print("\nDistress correlation on Training Data: \n")
for i in range(train_personality.shape[1]):
    print("NRC feature {} correlation: {}".format(personality_features[i], 
                                                  pearsonr(column(train_personality, i), train_distress)))
    
print("\nDistress correlation on Dev Data: \n")
for i in range(dev_personality.shape[1]):
    print("NRC feature {} correlation: {}".format(personality_features[i], 
                                                  pearsonr(column(dev_personality, i), dev_distress)))


Distress correlation on Training Data: 

NRC feature personality_conscientiousness correlation: (-0.05153061053217745, 0.026257626702303652)
NRC feature personality_openess correlation: (0.014207526470026804, 0.5403014682871335)
NRC feature personality_extraversion correlation: (0.12405604614928988, 7.98785639534921e-08)
NRC feature personality_agreeableness correlation: (0.1265897305069749, 4.303881683442054e-08)
NRC feature personality_stability correlation: (-0.0837787455455481, 0.0002979021594045739)

Distress correlation on Dev Data: 

NRC feature personality_conscientiousness correlation: (-0.09324355779674652, 0.12642042352162314)
NRC feature personality_openess correlation: (0.10025728826674211, 0.10019756302891678)
NRC feature personality_extraversion correlation: (-0.25778449051654345, 1.7957123002921912e-05)
NRC feature personality_agreeableness correlation: (0.088025115396524, 0.1491638812272802)
NRC feature personality_stability correlation: (-0.35599698930373724, 1.73465

# High Empathy Analysis

In [51]:
he_train_data = train_data[train_data.gold_empathy_bin == 0].copy(deep=True)
print(he_train_data.shape)
he_dev_data = dev_data[dev_data.gold_empathy_bin == 0].copy(deep=True)
print(he_dev_data.shape)

he_train_empathy = he_train_data.gold_empathy.values
he_dev_empathy = he_dev_data.gold_empathy.values

(944, 224)
(150, 219)
